In [1]:
import pandas as pd
import numpy as np
import deepchem as dc
import warnings
warnings.filterwarnings('ignore')
from deepchem.models.torch_models import AttentiveFPModel
from deepchem.models.torch_models import GCNModel
from deepchem.metrics.metric import Metric
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt 
from Featurizer import Featurizer
from Predictions import Predict

2025-03-04 12:02:05.628873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.


In [2]:
data = pd.read_csv('PATH_TO_YOUR_DATA.csv')

# GCN

In [3]:
model = Featurizer(model= "gcn")
data_path = "PATH_TO_YOUR_DATA.csv"
test_dataset_gcn, transformers_gcn= model.featurize(data_path)
test_dataset_gcn_y= test_dataset_gcn.y

In [4]:
#Best Hyperparameters from Suuplemantary File S1
model_gcn = GCNModel(1,graph_conv_layers= [512,128,64], mode='classification',batch_size=64,learning_rate=0.001,dropout=0.15,param_hidden_feat = 128) 
model_gcn.restore('GCN_COVID-19_1:10.pt')

In [5]:
metric= [dc.metrics.roc_auc_score, dc.metrics.accuracy_score, dc.metrics.f1_score, dc.metrics.recall_score]
test_scores = {}
test_scores["GCN"] = model_gcn.evaluate(test_dataset_gcn, metric)

In [6]:
m =  Predict (model_name = "GCN")
df_gcn = m.predictions(model_gcn, data, test_dataset_gcn, test_dataset_gcn_y,test_scores)

# AFP

In [7]:
model = Featurizer(model= "afp")
data_path = "PATH_TO_YOUR_DATA.csv"
test_dataset_afp, transformers_afp= model.featurize(data_path)
test_dataset_afp_y= test_dataset_afp.y

In [8]:
#Best Hyperparameters from Suuplemantary File S1
model_afp =  AttentiveFPModel(n_tasks= 1, mode="classification", batch_size=32, learning_rate=0.001, droupout=0.1, num_timesteps=2, num_layers= 3) 
model_afp.restore('AFP_COVID-19_1:10.pt')

In [9]:
metric= [dc.metrics.roc_auc_score, dc.metrics.accuracy_score, dc.metrics.f1_score, dc.metrics.recall_score]
test_scores = {}
test_scores["AFP"] = model_afp.evaluate(test_dataset_afp, metric)

In [10]:
m =  Predict (model_name = "AFP")
df_2 = m.predictions(model_afp,data, test_dataset_afp,test_dataset_afp_y,test_scores)